In [80]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings(action='ignore')


## raw data 와 valid data 개요

In [81]:
raw_data = pd.read_csv("../data/sea/sea_managing_raw.csv", encoding='cp949')
print(raw_data.shape)
raw_data.head()

(1905, 37)


,국가코드,DB종류,특허/실용 구분,문헌종류 코드,발명의 명칭,요약,대표청구항,청구항 수,출원번호,출원일,...,Original CPC Main,Original CPC All,Original IPC Main,Original IPC All,Original US Class Main[US],Original US Class All[US],Original FI[JP],Original F-term[JP],Original Theme Code [JP],WIPS ON key
0,US,US,P,B2,Turboengine water wash system,A system for cleaning gas turbine engines is d...,1. An apparatus for cleaning wing mounted gas ...,64,11/644784,2006-12-22,...,NaN,NaN,B08B-009/00,B08B-009/00,134/166.R,134/166.R | 134/138,NaN,NaN,NaN,4.914000e+12
1,US,US,P,A1,TREATMENT OF TAILINGS WITH DEIONIZED SILICATE ...,A process for treating a tailings stream compr...,1. A process for treating a tailings stream co...,20,13/848244,2013-03-21,...,B09B-0003/0025,B09B-0003/0025,B09B-003/00,B09B-003/00,106/627,106/627 | 106/600 | 106/631 | 106/632 | 106/63...,NaN,NaN,NaN,5.414000e+12
2,US,US,P,A1,Method And Apparatus For Treating Tailings Usi...,There is a method described of treating tailin...,1. A method of treating tailings having a comb...,18,13/765924,2013-02-13,...,C02F-0001/48,C02F-0001/48,C02F-001/48,C02F-001/48,204/554,204/554,NaN,NaN,NaN,5.414000e+12
3,US,US,P,A1,METHOD FOR STABILIZING WASTE AND HAZARDOUS WASTE,The invention provides a process for stabilizi...,"1. A method for stabilizing hazardous waste, c...",20,13/767408,2013-02-14,...,A62D-0003/33,A62D-0003/33 | G21F-0009/302,A62D-003/33,A62D-003/33 | G21F-009/30,588/002,588/002 | 588/315,NaN,NaN,NaN,5.414000e+12
4,US,US,P,B2,Water treatment process and plant using ballas...,A method for treating wastewater using a balla...,1. A method of treating wastewater including: ...,18,12/521154,2007-12-26,...,NaN,NaN,C02F-001/52,C02F-001/52 | B01D-021/01 | B01D-021/02 | B01D...,210/709,210/709 | 210/711 | 210/712 | 210/713 | 210/71...,NaN,NaN,NaN,4.913050e+12


In [82]:
df = pd.DataFrame(raw_data, columns=raw_data.columns)

In [83]:
#검색식을 통해 수집한 유효특허 후보 데이터
val_data = pd.read_csv("../data/sea/sea_managing_val.csv", encoding = "cp949")
print(val_data.shape)
val_data.head()

(2098, 7)


,공개(등록)번호,공개(등록)일자,국가,구분,발명의 명칭,IPC,출원인
0,1014013000000,20140523,KR,등록,생태안착형 입체 호안구조 및 이의 시공방법,E02B,벽산건설 주식회사
1,1014013000000,20140523,KR,등록,생태안착형 입체 호안구조 및 이의 시공방법,E02B,벽산건설 주식회사
2,1013950840000,20140508,KR,등록,질소 및 인의 동시 제거가 가능한 생물여과 처리장치,C02F,주식회사 지승개발
3,1013950840000,20140508,KR,등록,질소 및 인의 동시 제거가 가능한 생물여과 처리장치,C02F,주식회사 지승개발
4,1013903570000,20140423,KR,등록,살수여상기능 및 정수처리기능을 갖는 자연형 하천 호안 공법,E02B,충북대학교 산학협력단


In [84]:
#미국 특허 확인
val_data.tail()

,공개(등록)번호,공개(등록)일자,국가,구분,발명의 명칭,IPC,출원인
2093,04126007,19781121,US,등록,Compaction of soil,E02D,L.B.
2094,04084381,19780418,US,등록,Stabilization of earth subsurface layers,E01B,Woodbine
2095,04084381,19780418,US,등록,Stabilization of earth subsurface layers,E01B,Woodbine
2096,03952522,19760427,US,등록,Irrigation systems automation,E02B,NaN
2097,03952522,19760427,US,등록,Irrigation systems automation,E02B,NaN


## raw data 필요없는 column 제거
* 미국 특허만 활용
* 일본 특허 정보 x  
ex) 'Original FI[JP]', 'Original F-term[JP]', 'Original Theme Code [JP]', '출원인 식별기호[JP]'

In [85]:
df.columns

Index(['국가코드', 'DB종류', '특허/실용 구분', '문헌종류 코드', '발명의 명칭', '요약', '대표청구항', '청구항 수',
       '출원번호', '출원일', '공개번호/공표/재공표 ', '공개일', '등록번호', '등록일', '출원인', '출원인 국적',
       '출원인 수', '출원인 대표명화 코드', '출원인 대표명화 영문명', '출원인 대표명화 국문명[KR]',
       '출원인 식별기호[JP]', '발명자/고안자', '우선권 번호', '우선권 국가', '우선권 주장일', '국제 공개번호',
       '국제 공개일', 'Original CPC Main', 'Original CPC All', 'Original IPC Main',
       'Original IPC All', 'Original US Class Main[US]',
       'Original US Class All[US]', 'Original FI[JP]', 'Original F-term[JP]',
       'Original Theme Code [JP]', 'WIPS ON key'],
      dtype='object')

In [86]:
df = raw_data.dropna(axis=1, how='all')
df.head()

,국가코드,DB종류,특허/실용 구분,문헌종류 코드,발명의 명칭,요약,대표청구항,청구항 수,출원번호,출원일,...,우선권 주장일,국제 공개번호,국제 공개일,Original CPC Main,Original CPC All,Original IPC Main,Original IPC All,Original US Class Main[US],Original US Class All[US],WIPS ON key
0,US,US,P,B2,Turboengine water wash system,A system for cleaning gas turbine engines is d...,1. An apparatus for cleaning wing mounted gas ...,64,11/644784,2006-12-22,...,NaN,NaN,NaN,NaN,NaN,B08B-009/00,B08B-009/00,134/166.R,134/166.R | 134/138,4.914000e+12
1,US,US,P,A1,TREATMENT OF TAILINGS WITH DEIONIZED SILICATE ...,A process for treating a tailings stream compr...,1. A process for treating a tailings stream co...,20,13/848244,2013-03-21,...,NaN,NaN,NaN,B09B-0003/0025,B09B-0003/0025,B09B-003/00,B09B-003/00,106/627,106/627 | 106/600 | 106/631 | 106/632 | 106/63...,5.414000e+12
2,US,US,P,A1,Method And Apparatus For Treating Tailings Usi...,There is a method described of treating tailin...,1. A method of treating tailings having a comb...,18,13/765924,2013-02-13,...,2012-07-09,NaN,NaN,C02F-0001/48,C02F-0001/48,C02F-001/48,C02F-001/48,204/554,204/554,5.414000e+12
3,US,US,P,A1,METHOD FOR STABILIZING WASTE AND HAZARDOUS WASTE,The invention provides a process for stabilizi...,"1. A method for stabilizing hazardous waste, c...",20,13/767408,2013-02-14,...,2012-07-02,NaN,NaN,A62D-0003/33,A62D-0003/33 | G21F-0009/302,A62D-003/33,A62D-003/33 | G21F-009/30,588/002,588/002 | 588/315,5.414000e+12
4,US,US,P,B2,Water treatment process and plant using ballas...,A method for treating wastewater using a balla...,1. A method of treating wastewater including: ...,18,12/521154,2007-12-26,...,2006-12-29,WO2008-083923,2008-07-17,NaN,NaN,C02F-001/52,C02F-001/52 | B01D-021/01 | B01D-021/02 | B01D...,210/709,210/709 | 210/711 | 210/712 | 210/713 | 210/71...,4.913050e+12


In [87]:
df.columns

Index(['국가코드', 'DB종류', '특허/실용 구분', '문헌종류 코드', '발명의 명칭', '요약', '대표청구항', '청구항 수',
       '출원번호', '출원일', '공개번호/공표/재공표 ', '공개일', '등록번호', '등록일', '출원인', '출원인 국적',
       '출원인 수', '출원인 대표명화 코드', '출원인 대표명화 영문명', '발명자/고안자', '우선권 번호', '우선권 국가',
       '우선권 주장일', '국제 공개번호', '국제 공개일', 'Original CPC Main', 'Original CPC All',
       'Original IPC Main', 'Original IPC All', 'Original US Class Main[US]',
       'Original US Class All[US]', 'WIPS ON key'],
      dtype='object')

## valid data에서 US data 외 데이터 drop

In [88]:
val_data = val_data[val_data.국가 == "US"]
val_data = val_data.drop_duplicates(val_data.columns)
val_data.shape
# val_data.to_csv("1.csv", encoding='cp949')

(75, 7)

## raw data와 valid data의 공개/등록 번호 form 맞춰주기

In [89]:
print("<valid> \n", val_data['공개(등록)번호'].head())
print("\n<raw> \n", raw_data['공개번호/공표/재공표 '].head(), "\n", raw_data['등록번호'].head())

<valid> 
 14    20140037379
31       08557121
61       08448279
78       08430600
96    20130028660
Name: 공개(등록)번호, dtype: object

<raw> 
 0    2008-0149141
1    2014-0007795
2    2014-0008229
3    2014-0005461
4    2010-0096335
Name: 공개번호/공표/재공표 , dtype: object 
 0    8628627
1        NaN
2        NaN
3        NaN
4    8617400
Name: 등록번호, dtype: object


In [90]:
df.loc[:, '공개번호/공표/재공표 '] = (df.loc[:, '공개번호/공표/재공표 '].str.split('-')).str.join('')
df.loc[:, '공개번호/공표/재공표 '].fillna(value = 0, inplace=True)

In [91]:
# 8자리 맞추기 ex)8704653 -> 08704653
# 00000000 -> 0

df.loc[:, '등록번호'] = df.loc[:, '등록번호'].apply(lambda x:'{0:0>8}'.format(x) if(np.all(pd.notnull(x))) else x).fillna(0)
for idx, i in enumerate(df['등록번호']):
    if str(i).isdigit() != True:
        df['등록번호'][idx] = 0

In [92]:
df['등록번호'].head(10)

0    08628627
1           0
2           0
3           0
4    08617400
5           0
6           0
7    08596928
8    08598405
9           0
Name: 등록번호, dtype: object

## valid patent 확인
US 유효특허 : 86개

In [93]:
#raw_data 중에서 val_data
y_idx_list = []
for idx, raw in enumerate(df["공개번호/공표/재공표 "]):
    for val in val_data["공개(등록)번호"]:
        if raw == val and idx not in y_idx_list:
            y_idx_list.append(idx)

for idx, raw in enumerate(df["등록번호"]):
    for val in val_data["공개(등록)번호"]:
        if raw == val and idx not in y_idx_list:
            y_idx_list.append(idx)

for idx, raw in enumerate(df["발명의 명칭"]):
    for val in val_data["발명의 명칭"]:
        if raw.lower() == val.lower() and idx not in y_idx_list:
            y_idx_list.append(idx)
y_data_list = [int(1) for i in range(len(y_idx_list))]
print(len(y_idx_list), len(y_data_list))

41 41


In [94]:
y_values = pd.Series(data=y_data_list, index=y_idx_list)
df.loc[:, "유효특허 여부"] = y_values
df["유효특허 여부"].fillna(0, inplace=True)

In [95]:
df['유효특허 여부'].value_counts()

0.0    1864
1.0      41
Name: 유효특허 여부, dtype: int64

<hr>

## IPC Digitization

In [96]:
df.loc[:, 'Original IPC Main'] = df.loc[:, 'Original IPC Main'].astype('str').str.slice(0,4)
df['Original IPC Main'].head()

0    B08B
1    B09B
2    C02F
3    A62D
4    C02F
Name: Original IPC Main, dtype: object

In [97]:
# |로 묶인 IPC 처리
ipc_idx_list = []
ipc_all = []
for idx, i in enumerate(df['Original IPC All']):
    i = i.split('|')
    if len(i) != 1: # 여러 개 있으면
        ipc_list = []
        for ipc in i:
            ipc_list.append((ipc.strip())[:4])
        ipc_all.append(('|').join(ipc_list))
        ipc_idx_list.append(idx)
    else:
        ipc_all.append((i[0].strip())[0:4])
        ipc_idx_list.append(idx)

In [98]:
ipc_values = pd.Series(data = ipc_all, index=ipc_idx_list)
df.loc[:, 'Original IPC All'] = ipc_values
df['Original IPC All'].head()

0                        B08B
1                        B09B
2                        C02F
3                   A62D|G21F
4    C02F|B01D|B01D|B01D|B01D
Name: Original IPC All, dtype: object

## fillna

In [99]:
df.loc[:, '출원인'] = df.loc[:, '출원인'].fillna(value='UKN')
df[['출원인']][60:70]

,출원인
60,INVENT UMWELT-UND VERFAHRENSTECHNIK AG
61,Kurt Ingerle
62,UKN
63,UKN
64,"Geopier Foundation Company, Inc."
65,UNIVERSITY OF SOUTH FLORIDA
66,UKN
67,Veolia Water Solutions & Technologies Support
68,"CH2M Hill, Inc."
69,UKN


## including dash/slash data preprocessing

In [100]:
df.loc[:, '출원일'] = (df.loc[:, '출원일'].str.split('-')).str.join('')
df.loc[:, '출원일'] = df.loc[:, '출원일'].fillna(0)

df.loc[:, '공개일'] = (df.loc[:, '공개일'].str.split('-')).str.join('')
df.loc[:, '공개일'] = df.loc[:, '공개일'].fillna(0)

df.loc[:, '등록일'] = (df.loc[:, '등록일'].str.split('-')).str.join('')
df.loc[:, '등록일'] = df.loc[:, '등록일'].fillna(0)

df.loc[:, '국제 공개번호'] = (df.loc[:, '국제 공개번호'].str.split('-')).str.join('').str.slice(2,-1)
df.loc[:, '국제 공개번호'] = (df.loc[:, '국제 공개번호'].str.split('/')).str.join('')
df.loc[:, '국제 공개번호'] = df.loc[:, '국제 공개번호'].fillna(value=int(0))

## Label Encoding

In [101]:
application_id = list(df['출원번호'])
le = LabelEncoder()
le.fit(application_id)

application_id_le = le.transform(application_id)
df['출원번호'] = application_id_le

In [102]:
patent_kind = list(df['문헌종류 코드'])
le = LabelEncoder()
le.fit(patent_kind)

patent_kind_le = le.transform(patent_kind)
df['문헌종류 코드'] = patent_kind_le

In [103]:
priority_id = list(df['우선권 번호'])
le = LabelEncoder()
le.fit(priority_id)

priority_id_le = le.transform(priority_id)
df['우선권 번호'] = priority_id_le

In [104]:
df = df.astype('str')

In [105]:
df.to_csv('../data/sea/data_sea_1.csv', encoding='cp949')

---------

In [106]:
df['patent index'] = [i for i in range(df.shape[0])]

## ipc main to EdgeList

In [107]:
# ipc 한 개
main_ipc_dict = {}
main_ipc_dict['patent'] = df['patent index']
main_ipc_dict['ipc'] = df['Original IPC Main']

main_ipc_edge = pd.DataFrame(main_ipc_dict)
main_ipc_edge.to_csv("../data/sea/sea_main_ipc_edge.tsv", sep='\t', index=False, header=True)

## ipc all to EdgeList

In [108]:
# ipc 여러 개
ipc_index = []
ipc = []
for idx, i in enumerate(df['Original IPC All']):
    i = i.split('|')
    if len(i) != 1:
        for j in range(len(i)):
            ipc_index.append(idx)
            ipc.append(i[j])
    else:
        ipc_index.append(idx)
        ipc.append(i[0])

In [109]:
ipc_idx_values = pd.Series(data=ipc_index)
ipc_values = pd.Series(data=ipc)

all_ipc_dict = {}
all_ipc_dict['ipc'] = ipc_values
all_ipc_dict['patent'] = ipc_idx_values

all_ipc_edge = pd.DataFrame(all_ipc_dict)
all_ipc_edge.to_csv("../data/sea/sea_all_ipc_edge.tsv", sep='\t', index=False, header=True)

## US Class main to EdgeList

In [110]:
# 한 개
main_usclass_dict = {}
main_usclass_dict['patent'] = df['patent index']
main_usclass_dict['us_class'] = df['Original US Class Main[US]']

main_usclass_edge = pd.DataFrame(main_usclass_dict)
main_usclass_edge.to_csv("../data/sea/sea_main_usclass_edge.tsv", sep='\t', index=False, header=True)

## US Class all to EdgeList

In [111]:
# 여러 개
usclass_index = []
usclass = []
for idx, i in enumerate(df['Original US Class All[US]']):
    i = i.split('|')
    if len(i) != 1:
        for j in range(len(i)):
            usclass_index.append(idx)
            usclass.append(i[j].strip())
    else:
        usclass_index.append(idx)
        usclass.append(i[0])

In [112]:
usclass_idx_values = pd.Series(data=usclass_index)
usclass_values = pd.Series(data=usclass)

all_usclass_dict = {}
all_usclass_dict['us_class'] = usclass_values
all_usclass_dict['patent'] = usclass_idx_values

all_usclass_edge = pd.DataFrame(all_usclass_dict)
all_usclass_edge.to_csv("../data/sea/sea_all_usclass_edge.tsv", sep='\t', index=False, header=True)

## assignee to EdgeList

In [113]:
assignee = pd.read_csv('../data/sea/assignee.tsv', sep='\t', encoding='utf-8')
assignee = assignee.drop('Unnamed: 0', axis=1)

In [114]:
# patent id 와 assignee id 리스트 생성
patent_id =[]
assignee_id_list = []
for i in range(len(assignee)):
    ids = assignee.fillna('0').iloc[[i]].values
    for j in ids[0]:
        patent_assignee = j.replace("(","").replace(")","").replace("'","").split(",")
        if len(patent_assignee) != 1:
            patent_id.append(patent_assignee[0].strip())
            assignee_id_list.append(patent_assignee[1].strip())

In [115]:
patent_values = pd.Series(data=patent_id)
assignee_values = pd.Series(data=assignee_id_list)

patent_assignee_dict= {}
patent_assignee_dict['patent_id'] = patent_id
patent_assignee_dict['assignee_id'] = assignee_id_list

patent_assignee_edge = pd.DataFrame(patent_assignee_dict)
patent_assignee_edge.to_csv('../data/sea/sea_assignee_edge.tsv', sep='\t', header=True)

## application citation to EdgeList

In [116]:
application_citation = pd.read_csv('../data/sea/application_citation.tsv', sep='\t', encoding='utf-8')
application_citation = application_citation.drop('Unnamed: 0', axis=1)
application_citation = application_citation.dropna(how='all')

In [117]:
patent_id =[]
application_citation_id_list = []
for i in range(len(application_citation)):
    ids = application_citation.fillna('0').iloc[[i]].values
    for j in ids[0]:
        patent_application_citation = j.replace("(","").replace(")","").replace("'","").split(",")
        if len(patent_application_citation) != 1:
            patent_id.append(patent_application_citation[0].strip())
            application_citation_id_list.append(patent_application_citation[1].strip())

In [118]:
patent_values = pd.Series(data=patent_id)
application_citation_values = pd.Series(data=application_citation_id_list)

patent_application_citation_dict= {}
patent_application_citation_dict['patent_id'] = patent_id
patent_application_citation_dict['application_citation_id'] = application_citation_id_list

patent_application_citation_edge = pd.DataFrame(patent_application_citation_dict)
patent_application_citation_edge.to_csv('../data/sea/sea_application_citation_edge.tsv', sep='\t', header=True)

## grant citation to EdgeList

In [119]:
grant_citation = pd.read_csv('../data/sea/grant_citation.tsv', sep='\t', encoding='utf-8')
grant_citation = grant_citation.drop('Unnamed: 0', axis=1)
grant_citation = grant_citation.dropna(how='all')

In [120]:
patent_id =[]
grant_citation_id_list = []
for i in range(len(grant_citation)):
    ids = grant_citation.fillna('0').iloc[[i]].values
    for j in ids[0]:
        patent_grant_citation = j.replace("(","").replace(")","").replace("'","").split(",")
        if len(patent_grant_citation) != 1:
            patent_id.append(patent_grant_citation[0].strip())
            grant_citation_id_list.append(patent_grant_citation[1].strip())

In [121]:
patent_values = pd.Series(data=patent_id)
grant_citation_values = pd.Series(data=grant_citation_id_list)

patent_grant_citation_dict= {}
patent_grant_citation_dict['patent_id'] = patent_id
patent_grant_citation_dict['grant_citation_id'] = grant_citation_id_list

patent_grant_citation_edge = pd.DataFrame(patent_grant_citation_dict)
patent_grant_citation_edge.to_csv('../data/sea/sea_grant_citation_edge.tsv', sep='\t', header=True)

## inventor to EdgeList

In [122]:
df.columns

Index(['국가코드', 'DB종류', '특허/실용 구분', '문헌종류 코드', '발명의 명칭', '요약', '대표청구항', '청구항 수',
       '출원번호', '출원일', '공개번호/공표/재공표 ', '공개일', '등록번호', '등록일', '출원인', '출원인 국적',
       '출원인 수', '출원인 대표명화 코드', '출원인 대표명화 영문명', '발명자/고안자', '우선권 번호', '우선권 국가',
       '우선권 주장일', '국제 공개번호', '국제 공개일', 'Original CPC Main', 'Original CPC All',
       'Original IPC Main', 'Original IPC All', 'Original US Class Main[US]',
       'Original US Class All[US]', 'WIPS ON key', '유효특허 여부', 'patent index'],
      dtype='object')

In [123]:
inventor_index = []
inventor = []
for idx, i in enumerate(df['발명자/고안자']):
    i = i.split('|')
    if len(i) != 1:
        for j in range(len(i)):
            inventor_index.append(idx)
            inventor.append(i[j].strip())
    else:
        inventor_index.append(idx)
        inventor.append(i[0])

In [124]:
inventor_idx_values = pd.Series(data=inventor_index)
inventor_values = pd.Series(data=inventor)

inventor_dict = {}
inventor_dict['patent_id'] = inventor_idx_values
inventor_dict['inventor'] = inventor_values

inventor_edge = pd.DataFrame(inventor_dict)
inventor_edge.to_csv('../data/sea/sea_inventor_edge.tsv', sep='\t', encoding='utf-8', header=True)

## applicant

In [125]:
applicant_index = []
applicant = []
for idx, i in enumerate(df['출원인']):
    i = i.split('|')
    if len(i) != 1:
        for j in range(len(i)):
            applicant_index.append(idx)
            applicant.append(i[j].strip().lower())
    else:
        applicant_index.append(idx)
        applicant.append(i[0].lower())

In [126]:
applicant_idx_values = pd.Series(data=applicant_index)
applicant_values = pd.Series(data=applicant)

applicant_dict = {}
applicant_dict['patent_id'] = applicant_idx_values
applicant_dict['applicant'] = applicant_values

applicant_edge = pd.DataFrame(applicant_dict)
applicant_edge.to_csv('../data/sea/sea_applicant_edge.tsv', sep='\t', encoding='utf-8', header=True)

----------

In [127]:
df = pd.read_csv("../data/sea/data_sea_1.csv", encoding='cp949')

In [128]:
print(df.shape)
df.columns

(1905, 34)


Index(['Unnamed: 0', '국가코드', 'DB종류', '특허/실용 구분', '문헌종류 코드', '발명의 명칭', '요약',
       '대표청구항', '청구항 수', '출원번호', '출원일', '공개번호/공표/재공표 ', '공개일', '등록번호', '등록일',
       '출원인', '출원인 국적', '출원인 수', '출원인 대표명화 코드', '출원인 대표명화 영문명', '발명자/고안자',
       '우선권 번호', '우선권 국가', '우선권 주장일', '국제 공개번호', '국제 공개일', 'Original CPC Main',
       'Original CPC All', 'Original IPC Main', 'Original IPC All',
       'Original US Class Main[US]', 'Original US Class All[US]',
       'WIPS ON key', '유효특허 여부'],
      dtype='object')

## convert column's name to english & drop columns

In [129]:
df = df.rename(index=str, 
               columns={'등록번호': 'grant_id','출원번호':'application_id', '공개번호/공표/재공표 ':'published_grant_id',
                        '발명의 명칭': "title", '대표청구항':'claim', '청구항 수':'n_claim', "발명자/고안자":"inventor",
                        '우선권 번호':'priority_id', '요약': 'abstract', 'Unnamed: 0' : 'index', '등록일':'grant_date',
                        '출원일':'application_date', '공개일':'published_date', '출원인':'applicant', 
                        '출원인 수':'n_applicant',
                        '출원인 대표명화 코드':'applicant_code', '국제 공개번호':'international_published_id', 
                        'Original IPC Main':'ipc_main', 'Original IPC All':'ipc_all',
                        'Original US Class Main[US]':'usclass_main', 'Original US Class All[US]':'usclass_all',
                        '유효특허 여부':'valid_patent', '문헌종류 코드':'patent_kind'})

In [130]:
df = df.drop(['특허/실용 구분', 'DB종류', '국가코드', '출원인 대표명화 영문명', '국제 공개일', '우선권 주장일',
             'Original CPC Main', 'Original CPC All', 'WIPS ON key', '출원인 국적', '우선권 국가'], axis=1)

In [131]:
df.shape

(1905, 23)

## merge edgelist to df

### assignee

In [132]:
assignee_edge = pd.read_csv("../data/sea/sea_assignee_edge.tsv", sep='\t')
for i, j in enumerate(df.grant_id):
    if j in list(assignee_edge['patent_id']):
        x = assignee_edge.loc[assignee_edge['patent_id']==j]['Unnamed: 0']
        assignee_edge.at[x,'patent_id']= i

In [133]:
cols = assignee_edge.columns.tolist()[::-1]
assignee_edge = assignee_edge[cols]

assignee_edge["value"] = 1
assignee_matrix = pd.pivot_table(assignee_edge, index=['patent_id', 'assignee_id'])

assignee_matrix_reidx = assignee_matrix.reset_index()
assignee_matrix_reidx = assignee_matrix_reidx.pivot(values= "value",index="patent_id",columns="assignee_id").fillna(0)

df.index = df['index']
df = df.drop_duplicates()
df = df.join(assignee_matrix_reidx, how='outer')
df.shape

(1905, 284)

### appication citation

In [134]:
application_citation_edge = pd.read_csv("../data/sea/sea_application_citation_edge.tsv", sep='\t')

for i, j in enumerate(df.grant_id):
    if j in list(application_citation_edge['patent_id']):
        x = application_citation_edge.loc[application_citation_edge['patent_id']==j]['Unnamed: 0']
        application_citation_edge.at[x,'patent_id']= i

In [135]:
cols = application_citation_edge.columns.tolist()[::-1]
application_citation_edge = application_citation_edge[cols]

application_citation_edge["value"] = 1
application_citation_matrix = pd.pivot_table(application_citation_edge, index=['patent_id', 'application_citation_id'])

application_citation_matrix_reidx = application_citation_matrix.reset_index()
application_citation_matrix_reidx = application_citation_matrix_reidx.pivot(values= "value",index="patent_id",columns="application_citation_id").fillna(0)

df.index = df['index']
df = df.drop_duplicates()
df = df.join(application_citation_matrix_reidx, how='outer')
df.shape

(1905, 1016)

### grant citaion

In [136]:
grant_citation_edge = pd.read_csv("../data/sea/sea_grant_citation_edge.tsv", sep='\t')

for i, j in enumerate(df.grant_id):
    if j in list(grant_citation_edge['patent_id']):
        x = grant_citation_edge.loc[grant_citation_edge['patent_id']==j]['Unnamed: 0']
        grant_citation_edge.at[x,'patent_id']= i

In [137]:
cols = grant_citation_edge.columns.tolist()[::-1]
grant_citation_edge = grant_citation_edge[cols]
grant_citation_edge["value"] = 1
grant_citation_matrix = pd.pivot_table(grant_citation_edge, 
                                             index=['patent_id', 'grant_citation_id'])

grant_citation_matrix_reidx = grant_citation_matrix.reset_index()
grant_citation_matrix_reidx = grant_citation_matrix_reidx.pivot(values= "value",index="patent_id",columns="grant_citation_id").fillna(0)
df.index = df['index']
df = df.drop_duplicates()
df = df.join(grant_citation_matrix_reidx, how='outer')
df.shape

(1905, 7710)

### ipc all

In [138]:
all_ipc_edge = pd.read_csv("../data/night/night_all_ipc_edge.tsv", sep='\t')
cols = all_ipc_edge.columns.tolist()[::-1]

In [139]:
all_ipc_edge = all_ipc_edge[cols]
all_ipc_edge["value"] = 1
all_ipc_matrix = pd.pivot_table(all_ipc_edge, index=['patent', 'ipc'])
all_ipc_matrix.head()

value
patent ipc        
0      B23P      1
1      G01S      1
2      B60Q      1
       G08G      1
3      B23K      1

In [140]:
all_ipc_matrix_reidx = all_ipc_matrix.reset_index()
all_ipc_matrix_reidx = all_ipc_matrix_reidx.pivot(index="patent",columns="ipc")["value"].fillna(0)
all_ipc_matrix_reidx.head()

ipc,A01B,A01C,A01G,A01N,A21B,A24B,A24F,A41G,A42B,A45B,...,H03K,H04B,H04K,H04L,H04M,H04N,H04Q,H04W,H05B,H05K
patent,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
df = df.drop('ipc_all', axis=1)
df = df.drop('ipc_main', axis=1)

df.index = df['index']
df = df.drop_duplicates()
df = df.merge(all_ipc_matrix_reidx, left_index=True, right_index=True)
print(df.shape)

(1236, 7907)


### usclass all

In [142]:
all_usclass_edge = pd.read_csv("../data/night/night_all_usclass_edge.tsv", sep='\t')
cols = all_usclass_edge.columns.tolist()[::-1]

all_usclass_edge = all_usclass_edge[cols]
all_usclass_edge["value"] = 1
all_usclass_matrix = pd.pivot_table(all_usclass_edge, index=['patent', 'us_class'])

all_usclass_matrix_reidx = all_usclass_matrix.reset_index()
all_usclass_matrix_reidx = all_usclass_matrix_reidx.pivot(index="patent",columns="us_class")["value"].fillna(0)
df = df.drop('usclass_all', axis=1)
df = df.drop('usclass_main', axis=1)

df.index = df['index']
df = df.drop_duplicates()
df = df.merge(all_usclass_matrix_reidx, left_index=True, right_index=True)
print(df.shape)

(1236, 10004)


### inventor

In [143]:
inventor_edge = pd.read_csv("../data/night/night_inventor_edge.tsv", sep='\t')
cols = inventor_edge.columns.tolist()[::-1]

inventor_edge = inventor_edge[cols]
inventor_edge["value"] = 1
inventor_matrix = pd.pivot_table(inventor_edge, index=['patent_id', 'inventor'])

inventor_matrix_reidx = inventor_matrix.reset_index()
inventor_matrix_reidx = inventor_matrix_reidx.pivot(index="patent_id",columns="inventor")["value"].fillna(0)

df = df.drop('inventor', axis=1)
df.index = df['index']
df = df.drop_duplicates()
df = df.merge(inventor_matrix_reidx, left_index=True, right_index=True)
print(df.shape)

(1236, 12521)


### applicant

In [144]:
applicant_edge = pd.read_csv("../data/night/night_applicant_edge.tsv", sep='\t')
cols = applicant_edge.columns.tolist()[::-1]

applicant_edge = applicant_edge[cols]
applicant_edge["value"] = 1
applicant_matrix = pd.pivot_table(applicant_edge, index=['patent_id', 'applicant'])

applicant_matrix_reidx = applicant_matrix.reset_index()
applicant_matrix_reidx = applicant_matrix_reidx.pivot(index="patent_id",columns="applicant")["value"].fillna(0)

df = df.drop('applicant', axis=1)
df = df.drop('applicant_code', axis=1)

df.index = df['index']
df = df.drop_duplicates()
df = df.merge(applicant_matrix_reidx, left_index=True, right_index=True)
print(df.shape)

(1236, 13127)


In [145]:
df.to_csv("../data/night/night_dataset_1.csv", index=False, header=True)

In [146]:
set(df.columns) - set(df.describe().columns)

{'abstract', 'claim', 'title'}

----------

In [153]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [159]:
df.drop(['title', 'abstract', 'claim'], axis=1, inplace=True)

x = df.loc[:, df.columns!='valid_patent'].astype(float)
y = df['valid_patent'].astype(int)

## Train/Test Split

In [160]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=50)

## SMOTE

In [161]:
print('Original dataset shape {}'.format(Counter(y_train)))
sm = SMOTE(random_state=42)
x_res , y_res = sm.fit_resample(x_train, y_train)
print('Resampled dataset shape {}'.format(Counter(y_res)))
print('Y test dataset shpae {}'.format(Counter(y_test)))

Original dataset shape Counter({0: 843, 1: 22})
Resampled dataset shape Counter({0: 843, 1: 843})
Y test dataset shpae Counter({0: 364, 1: 7})


## Model

### Logistic Regression

In [162]:
from sklearn import linear_model, metrics
from sklearn.metrics import classification_report, confusion_matrix

In [163]:
logreg = linear_model.LogisticRegression(fit_intercept=False)
logreg.fit(x_res, y_res)

result = logreg.predict(x_test)

print("This classification report of using threshold: \n",classification_report(y_test, result))
print ("This is accuracy score:",metrics.accuracy_score(y_test, result),"\n")

This classification report of using threshold: 
               precision    recall  f1-score   support

           0       0.98      0.87      0.92       364
           1       0.02      0.14      0.04         7

    accuracy                           0.86       371
   macro avg       0.50      0.51      0.48       371
weighted avg       0.96      0.86      0.91       371

This is accuracy score: 0.8571428571428571 



In [164]:
prob = logreg.predict_proba(x_test)

y_pred = np.zeros(len(x_test))
y_pred[prob[:,1] > 0.35] = 1

print("This classification report of using threshold: \n",classification_report(y_test, y_pred))
print ("This is accuracy score:",metrics.accuracy_score(y_test, y_pred),"\n")

This classification report of using threshold: 
               precision    recall  f1-score   support

           0       0.96      0.14      0.24       364
           1       0.02      0.71      0.03         7

    accuracy                           0.15       371
   macro avg       0.49      0.43      0.14       371
weighted avg       0.94      0.15      0.24       371

This is accuracy score: 0.1509433962264151 



### Decision Tree

In [165]:
from sklearn import tree

dt = tree.DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_leaf=5)
dt.fit(x_res, y_res)

result = dt.predict(x_test)

print("This classification report of using threshold: \n",classification_report(y_test, result))
print ("This is accuracy score:",metrics.accuracy_score(y_test, result),"\n")

This classification report of using threshold: 
               precision    recall  f1-score   support

           0       0.99      0.89      0.93       364
           1       0.07      0.43      0.12         7

    accuracy                           0.88       371
   macro avg       0.53      0.66      0.53       371
weighted avg       0.97      0.88      0.92       371

This is accuracy score: 0.8787061994609164 



### Random Forest

In [171]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42, max_depth=5)
rf.fit(x_res, y_res)

result = rf.predict(x_test)

print("This classification report of using threshold: \n",classification_report(y_test, result))
print ("This is accuracy score:",metrics.accuracy_score(y_test, result),"\n")

This classification report of using threshold: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       364
           1       1.00      0.14      0.25         7

    accuracy                           0.98       371
   macro avg       0.99      0.57      0.62       371
weighted avg       0.98      0.98      0.98       371

This is accuracy score: 0.9838274932614556 

